In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from kokoro import KModel
model = KModel(None, None).eval()



/Users/adrianlyjak/dev/onnx/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/Users/adrianlyjak/dev/onnx/.venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [3]:
import time
import torch

# Constants from the model
batch_size = 1
context_length = 512  # Based on position embeddings size
style_dim = 256      # Total style dimension (128 + 128) based on ref_s splitting

# Create dummy inputs
# 1. phonemes: str - but we'll skip this for ONNX export and start with input_ids
dummy_seq_length = 10  # Example length, must be <= context_length-2
input_ids = torch.LongTensor([[0] + [1]*dummy_seq_length + [0]])  # [batch_size, seq_length]

# 2. ref_s: style reference tensor
ref_s = torch.randn(batch_size, style_dim)  # [batch_size, 256]

# Export to ONNX
dummy_inputs = (input_ids, ref_s)

start_time = time.time()
print("trying dummy inputs")
model(input_ids, ref_s)
print(f"time for dummy inputs: {time.time() - start_time}")


trying dummy inputs
time for dummy inputs: 0.0977940559387207


In [4]:
# import torch
# traced = torch.jit.trace(
#     model,
#     example_inputs=dummy_inputs
    
# )

In [ ]:
import onnx
import torch

# Wrap the export in timing and debug code
def export_with_debug(model, dummy_inputs, path):
    print("Starting ONNX export...")
    start_time = time.time()
    
    try:
        torch.onnx.export(
            model,
            dummy_inputs,
            "kokoro.onnx",
            opset_version=11,  # or 15
            do_constant_folding=False,
            operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
            verbose=True,
            training=torch.onnx.TrainingMode.EVAL,
        )
        print(f"Export completed in {time.time() - start_time:.2f} seconds")
        
        # Try to load and check the model
        print("Validating exported model...")
        model = onnx.load(path)
        onnx.checker.check_model(model)
        print("Model validation successful")
        
    except Exception as e:
        print(f"Export failed after {time.time() - start_time:.2f} seconds")
        print(f"Error: {str(e)}")
        raise

# Try the export
dummy_seq_length = 5
input_ids = torch.LongTensor([[0] + [1]*dummy_seq_length + [0]])
ref_s = torch.randn(1, 256)

export_with_debug(model, (input_ids, ref_s), "kokoro.onnx")

Starting ONNX export...


/Users/adrianlyjak/dev/onnx/.venv/lib/python3.12/site-packages/torch/onnx/utils.py:478: FutureWarning: Setting `operator_export_type` to something other than default is deprecated. The option will be removed in a future release.
  warnings.warn(
/Users/adrianlyjak/dev/onnx/.venv/lib/python3.12/site-packages/kokoro/model.py:82: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  input_ids = list(filter(lambda i: i is not None, map(lambda p: self.vocab.get(p), phonemes)))
/Users/adrianlyjak/dev/onnx/.venv/lib/python3.12/site-packages/kokoro/modules.py:157: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  i

Torch IR graph at exception: graph(%phonemes : Long(1, 7, strides=[7, 1], requires_grad=0, device=cpu),
      %1 : Float(1, 256, strides=[256, 1], requires_grad=0, device=cpu),
      %2 : Long(requires_grad=0, device=cpu),
      %3 : Bool(requires_grad=0, device=cpu),
      %bert.embeddings.word_embeddings.weight : Float(178, 128, strides=[128, 1], requires_grad=1, device=cpu),
      %bert.embeddings.position_embeddings.weight : Float(512, 128, strides=[128, 1], requires_grad=1, device=cpu),
      %bert.embeddings.token_type_embeddings.weight : Float(2, 128, strides=[128, 1], requires_grad=1, device=cpu),
      %bert.embeddings.LayerNorm.weight : Float(128, strides=[1], requires_grad=1, device=cpu),
      %bert.embeddings.LayerNorm.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %bert.encoder.embedding_hidden_mapping_in.weight : Float(768, 128, strides=[128, 1], requires_grad=1, device=cpu),
      %bert.encoder.embedding_hidden_mapping_in.bias : Float(768, strides=[1

KeyboardInterrupt: 